In [ ]:
from IPython.core.display import HTML

def css_styling():
    styles = open("styles/workshop.css", "r").read()
    return HTML(styles)
css_styling()

<div class="alert alert-warning">
<h2>Goals</h2>
</div>

* To demonstate the processing of a large data set with Python and R.
* To practice data analysis tasks in Python and R.

In this first capstone project, we will be processing a large set of NOAA weather data from their [Integrated Surface Global Hourly Data](https://data.noaa.gov/dataset/integrated-surface-global-hourly-data).  

Some relevant information from the NOAA website on this data set:

>Integrated Surface Data (ISD) is digital data set DSI-3505, archived at the National Climatic Data Center (NCDC). The Integrated Surface Data (ISD) database is composed of worldwide surface weather observations from over 20,000 stations, collected and stored from sources such as the Automated Weather Network (AWN), the Global Telecommunications System (GTS), the Automated Surface Observing System (ASOS), and data keyed from paper forms. Most digital observations are decoded either at operational centers and forwarded to the Federal Climate Complex (FCC) in Asheville, NC, or decoded at the FCC. The US Air Force Combat Climatology Center (AFCCC), the National Climatic Data Center (NCDC), and the US Navy's Fleet Numerical Meteorological and Oceanographical Command Detachment (FNMOD), make up the FCC in Asheville. Each agency is responsible for data ingest, quality control, and customer support for surface climatological data. All data are now stored in a single ASCII format. 

> [...]

> The data are sorted by station-year-month-day-hour-minute. Parameters included are: air quality, atmospheric pressure, atmospheric temperature/dew point, atmospheric winds, clouds, precipitation, ocean waves, tides and more.

If you click on the [FTP download link](ftp://ftp.ncdc.noaa.gov/pub/data/noaa), you can even download the data yourself if you like (don't do it now, though;  you'll be at it for several days).  However, for this project we've done a fair bit of work for you:  we've downloaded every weather station recording from the NOAA data set going back to 1901 (180 gigabytes of data *when compressed*), compiled the data into yearly digests, and drawn a sample of 10000 records from each year.  

In the data directory there should be one sample file for each year from 1901-2014:

In [ ]:
!ls data

---

#### Task

Your first task will be to take a look at these files.  List the first 5 lines of the file 1958.sample at the command line.

---

You will notice that the data consists of a series of line, one per observation from each weather station.  They look like this (accounting for line wrapping in your browser window):
    
> 0377999999129261958010310005+27683-097283SAO  +0006NGP  V02004559009352200059N024000599+01225+00895102315ADDGA1999+999999004GA2999+999999999GA3999+999999999GA4999+999999999GD12035+9999999GD20005+9999999GD30005+9999999GD40005+9999999GF103035999999999999999999MA1999999102245EQDN01 00000JPWTH 1QNNE11 1 00204E11 1 00000E11 1 00000E11 1 00000G11U1 99999H11C1 32999H11 1 00999H11 1 00999H11 1 00999K11 1 00048L11 1 01500M11 1 30190N11 1 00000Q11 1 10231S11 1 00054V11 1 00404W11 1 22018

<div class="alert alert-info">
In this project, we will be analyzing information about the *yearly minimum temperature* recorded globally. Your job will be to extract the pertinent data from the sampled data files and process them into a smaller data set.  You will do this by writing a Python script that will load each sample file, process the lines and retrieve temperature and other data from each line, find the yearly minimum temperature recorded from all of those lines, and then group the minimum temperature information into another file for analysis in R.
</div>

By the end, you will have analysed the entire NOAA database and, among other analyses, made a plot that looks like this:

![Map plot](images/minimum_temperature_map_plot.png)

<div class="alert alert-warning">When approaching a project like this, our first goal is to break the problem down into smaller units that we can build up into a larger analysis.  The natural unit in this problem is the *line*:  each line of the sample files consists of a single observation from a weather station, which we can process to get useful information out of.</div>

---

#### Task

We will start by writing three functions, to retrieve the temperature, timestamp, and position from each line.  We've provided you with the skeleton of each function and some helpful information to get you started, as well as sample output that your function should return so that you can make sure you're doing it correctly.  Replace the ```'pass' ``` section of each function with your code, execute each cell to load it into memory, and then execute the test cell at the bottom to make sure that your functions are returning the correct information.  In the test_line cell, I have also provided a break down of some of the information contained in each line (this information is lifted from p. 18 of *Hadoop: The Definitive Guide*, by Tom White, published by O'Reilly).

---

In [1]:
# 0057
# 332130 # USAF weather station identifier 
# 99999 # WBAN weather station identifier 19500101 # observation date
# 0300 # observation time
# 4
# +51317 # latitude (degrees x 1000) 
# +028783 # longitude (degrees x 1000) 
# FM-12
# +0171 # elevation (meters)
# 99999
# V020
# 320 # wind direction (degrees)
# 1 # quality code
# N
# 0072
# 1
# 00450 # sky ceiling height (meters)
# 1 # quality code
# C
# N
# 010000 # visibility distance (meters)
# 1
# N
# 9 -0128 1 -0139 1 10268 1
# # quality code
# # air temperature (degrees Celsius x 10)
# # quality code
# # dew point temperature (degrees Celsius x 10) # quality code
# # atmospheric pressure (hectopascals x 10)
# # quality code

test_line = "0220010010999992001010106004+70930-008660FM-12+0009ENJA V0200601N006710090019N0100001N1-00451-00951100251ADDAA199000091AY121061AY221061GF108991081081004501999999KA1120N-00451MA1999999100141MD1310031+9999MW1031REMSYN088AAXX  01064 01001 31460 80613 11045 21095 30014 40025 53003 70322 888// 333 21045 91117;EQDQ01  00002PRCP99"

In [2]:
def temperature(line):
    """Return the temperature in Celsius, which is between positions 87 and 92 of the observation.  The masurement is
        an integer Celsius value multiplied by 10 (i.e. '-0045', which is -4.5 degrees Celsius) and must be converted.  """
    return float(line[87:92])/10

In [3]:
def timestamp(line):
    """Returns a tuple with the integer year, month and day values of the observation. These are at indices 15-18, 19-20, 21-22, and 23-26."""
    return (int(line[15:19]),int(line[19:21]),int(line[21:23]))

In [4]:
def position(line):
    """Return a tuple with the float latitude and longitude of the observation.  These are at 28:33 and 34:40 of the line respectively.
    The latitude and longitude values are in degrees and multiplied by 1000;  for example, +70930 is 70.93 degrees Celsius."""
    return (float(line[28:34])/1000, float(line[34:41])/1000)


In [5]:
print temperature(test_line) # Should return: -4.5
print timestamp(test_line) # Should return (2001,1,1)
print position(test_line) # Should return (70.93, -8.66)

-4.5
(2001, 1, 1)
(70.93, -8.66)


--- 

#### Task

Now that we can process lines, we need do so something with that.  A sample file is composed of lines, so our next job will be to process an sample file in order retrieve useful information.  

You will write a function that takes a filename as a parameter and returns a dictionary with six pieces of information from each file.  Here are the keys for the dictionary:

* 'temperature': The minimum temperature in that year.  
* 'year': the year of the file you processed.
* 'month': the month of the minimum recorded temperature.
* 'day': the day of the minimum recorded temperature.
* 'latitude': the latitude of the station.
* 'longitude': the longitude of the station.

The skeleton of the function is given below, along with a test cell and a line of sample output so that you can determine if you've written the function correctly.

---

In [8]:
def yearly_minimum(filename):
    """ Processes a file of NOAA weather data and returns a dictionary with five keys: 'temperature', 'year','month', 'day', 'latitude', and 'longitude'. """
    with open(filename,"r") as infile:
        min_temp = 99999
        min_timestamp = None
        min_position = None

        for line in infile.readlines():
            temp = temperature(line)
            lat, lon = position(line)
            if temp < min_temp and (-90 < lat < 90) and (-180 < lon < 180):
                min_temp = temp
                year, month, day = timestamp(line)
                min_lat, min_lon = lat,lon

        return {'temperature':min_temp, 'year':year, 'month':month, 'day':day, 'latitude':min_lat, 'longitude':min_lon}


In [9]:
print yearly_minimum('data/1958.sample')
# Sample output:  {'temperature': -52.8, 'longitude': 121.617, 'month': 2, 'year': 1958, 'latitude': 63.767, 'day': 1}

{'temperature': -52.8, 'longitude': 121.617, 'month': 2, 'year': 1958, 'latitude': 63.767, 'day': 1}


By now, you might see where we're going with this.  We can process lines, then files, and now we have an entire directory of files.  So let's process them!

---

#### Task

Your job now is to write a function that will process the ```data``` directory and return a **list of dictionaries**;  that would be a list of the return values from ```yearly_minimum``` for sample file in ```data```.  

The output will look like:

    [{'temperature': -19.1, 'longitude': -8.66, 'month': 3, 'year': 2001, 'latitude': 70.93, 'day': 2}, {'temperature': -33.3, 'longitude': 30.65, 'month': 2, 'year': 1901, 'latitude': 62.167, 'day': 16}, {'temperature': -32.8, 'longitude': 22.617, 'month': 3, 'year': 1902, 'latitude': 61.183, 'day': 12} ... ]
    

---



In [14]:
import glob

def get_minimums():
    minimums = []
    for fn in glob.glob("data/*.sample"):
        minimums.append(yearly_minimum(fn))
    return minimums

minimums = get_minimums()


Now that we have the data in a usable format, our last job will be to pass that data to R.  You could get fancy with this, but the simplest and most robust way is to simmply write out a .CSV file that you will then read in with R.  To do that, we will use Python's csv module, which we have not yet dealt with in this course.

---

#### Task

Use the [csv documentation](https://docs.python.org/2/library/csv.html) and/or Google to determine how to write the list of dictionaries you have created to a .CSV file named ```min_temp_readings.csv```.  

Hint:  the csv module has a facility explicitly meant to handle dictionaries as rows.  

---

In [15]:
import csv
with open('min_temp_readings.csv','w') as outfile:
    writer = csv.DictWriter(outfile,fieldnames=minimums[0].keys())
    writer.writeheader()
    writer.writerows(minimums)

With that finished, you've now completed the Python section of this project!  Now we will move on to to the R section of the project:  investigating and visualising our data.